In [2]:
!pip install chromedriver-autoinstaller

  Obtaining dependency information for chromedriver-autoinstaller from https://files.pythonhosted.org/packages/b7/17/824a202a4fdb6edda587f25878552beb0da6de70197d5f0b77b8d3781e91/chromedriver_autoinstaller-0.6.2-py3-none-any.whl.metadata


In [10]:
chromedriver_autoinstaller.install()

In [57]:
import requests
import os
import random
import time
import json

import pandas as pd
import numpy as np

from tqdm import tqdm

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import chromedriver_autoinstaller
import chromedriver_binary



In [11]:


def open_browser():
    """
    Opens a new automated browser window with all tell-tales of automated browser disabled
    """
    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized")
    
    # remove all signs of this being an automated browser
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)

    # open the browser with the new options
    driver = webdriver.Chrome(options=options)
    return driver

In [12]:
driver = open_browser()

In [13]:
def load_walmart(driver, attempts = 5):
    
    for attempt in range(0, attempts):
        # load the website
        driver.get('https://www.walmart.com')

        # make sure we wait with searching until the actual search bar appeared
        wait = 10
        try:
            search_box = WebDriverWait(driver, wait).until(EC.presence_of_element_located((By.CLASS_NAME, 'search-bar')))
            # no exception is thrown which means the wait is over and we found the search bar. We can finish this function
            return 
        except:
            # Since we're here, the search bar was not found. We just go back to the top of the for loop again
            continue

In [14]:
load_walmart(driver)

In [15]:
def search(query, driver):
    search_box = driver.find_element(By.CLASS_NAME, 'search-bar')
    for i in range(100):
        search_box.send_keys(Keys.BACKSPACE)
    search_box.send_keys(query + Keys.ENTER)

In [16]:
def get_suggestions(query):
    headers = {
        'authority': 'www.walmart.com',
        'accept': 'application/json',
        'accept-language': 'en-US',
        'device_profile_ref_id': 'LqIurHOiqfXM8Htfp_F9D8H7SeOsKpC8j3tj',
        'downlink': '10',
        'dpr': '2',
        'referer': 'https://www.walmart.com/',
        'sec-ch-ua': '"Google Chrome";v="119", "Chromium";v="119", "Not?A_Brand";v="24"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'traceparent': '00-b0ceab7f1e19f66d6cf2aa398fbd522a-95e65dbeedb13e57-00',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
        'wm_mp': 'true',
        'wm_qos.correlation_id': '7IMIWhZT7hpkZp3fl8paHlrYnJqgGqRXYLcP',
        'x-enable-server-timing': '1',
        'x-latency-trace': '1',
        'x-o-bu': 'WALMART-US',
        'x-o-correlation-id': '7IMIWhZT7hpkZp3fl8paHlrYnJqgGqRXYLcP',
        'x-o-mart': 'B2C',
        'x-o-platform': 'rweb',
        'x-o-platform-version': 'us-web-1.104.1-fe2ff4adf0765367f55d20cf86cfa7fcc85ec064-1101',
        'x-o-segment': 'oaoh',
    }

    params = {
        'term': query,
        'num': '9',
    }

    response = requests.get('https://www.walmart.com/typeahead/v3/complete', params=params, cookies={}, headers=headers)
    response = response.json()
    suggestions = response['queries']
    processed_suggestions = []
    
    for suggestion in suggestions:
        processed_suggestions.append(suggestion['displayName'])
        
    return processed_suggestions

In [17]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'
queries = []
for letter in tqdm(alphabet):
    suggestions = get_suggestions(letter)
    queries += suggestions


100%|██████████| 26/26 [00:06<00:00,  3.98it/s]


In [82]:
all_prod = []

In [83]:
len(all_prod)

0

In [84]:
selected_queries = queries[50:100]
for query in tqdm(selected_queries):
    search(query, driver)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source)
    soup = soup.find(id="__next")
    products = soup.find_all('div', attrs={'class': 'mb0 ph1 pa0-xl bb b--near-white w-25'})
    i = 1
    for product in products:
        try:
            if product:    
                name = product.find('span',attrs={'data-automation-id': 'product-title'}).text
                price = product.find('div',attrs={'data-automation-id': 'product-price'}).find('span', attrs={'class':'w_iUH7'}).text
                price = price[price.find('$'):]
                sponsored = 'Sponsored' in product.text
                try:
                    product.find('div', attrs={'class': 'flex', 'class':'mt2'}).find('span', attrs={'class':'w_iUH7'}).text
                    review_stats = product.find('div', attrs={'class': 'flex', 'class':'mt2'}).find('span', attrs={'class':'w_iUH7'}).text
                    stars = float(review_stats[:review_stats.find(' out')])
                    review_count = (review_stats[review_stats.find("s. ")+3:])
                except AttributeError:
                    stars = np.nan
                    review_count = np.nan


                productData = {}
                productData['name'] = name
                productData['price'] = price
                productData['stars'] = stars
                productData['reviews'] = review_count
                productData['sponsored'] = sponsored
                productData['rank'] = i
                all_prod.append(productData)
                i += 1
        except:
            continue

In [85]:
len(all_prod)

2069

In [89]:
df = pd.DataFrame(all_prod).drop_duplicates(subset=['name', 'price', 'stars', 'reviews'])

In [90]:
df.to_csv('walmartData.csv')  